# 1. Vector: 사용자 정의 시퀀스형

# 2. Vector 버전 #1: Vector2d 호환

In [35]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>

    def __iter__(self):
        return iter(self._components)  # <2>

    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6>

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>

In [36]:
test = Vector([3.1, 4.2])

In [37]:
test

Vector([3.1, 4.2])

In [38]:
test.__repr__()

'Vector([3.1, 4.2])'

In [30]:
Vector(range(10))

In [31]:
Vector((3, 4, 5))

In [32]:
test.__iter__()

In [39]:
reprlib.repr(list((3, 4, 5)))

'[3, 4, 5]'

# 3. 프로토콜과 덕 타이핑

In [41]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

# 4. Vector 버전 #2: 슬라이스 가능한 시퀀스

## 4.1 슬라이싱의 작동 방식

In [42]:
class MySeq:
    def __getitem__(self, index):
        return index

In [43]:
s = MySeq()

In [44]:
s[1]

1

In [63]:
s[2]

2

In [64]:
s[3]

3

In [45]:
s

In [46]:
s[1:4]

slice(1, 4, None)

In [47]:
dir(slice
)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [51]:
help(slice.indices)


Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [52]:
[1,2,3].[:]

SyntaxError: invalid syntax (3942304158.py, line 1)

In [56]:
l = [1,2,3]


In [58]:
l[1:2]

[2]

In [62]:
slice(1,5, None).indices(5)


(1, 5, 1)

## 4.2 슬라이스를 인식하는 __getitem__()

In [65]:
from array import array
import reprlib
import math
import numbers


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR_V2
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)  # <1>
        if isinstance(index, slice):  # <2>
            return cls(self._components[index])  # <3>
        elif isinstance(index, numbers.Integral):  # <4>
            return self._components[index]  # <5>
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))  # <6>
# END VECTOR_V2

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [66]:
v7 = Vector(range(7))

In [67]:
v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [68]:
v7[-1]

6.0